In [1]:
import openmeteo_requests
import polars as pl 
import datetime 
import matplotlib.pyplot as plt 
from openmeteo_sdk.Variable import Variable
from openmeteo_sdk.Aggregation import Aggregation
import requests_cache
from retry_requests import retry

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://ensemble-api.open-meteo.com/v1/ensemble"
params = {
	"latitude": 55.10091,
	"longitude": 14.70664,
	"hourly": ["precipitation", "rain"],
	"start_date": "2025-02-12",
	"end_date": "2025-02-13",
	"models": "icon_seamless"
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


# Process hourly data
hourly = response.Hourly()
hourly_variables = list(map(lambda i: hourly.Variables(i), range(0, hourly.VariablesLength())))
hourly_precipitation = filter(lambda x: x.Variable() == Variable.precipitation, hourly_variables)
hourly_rain = filter(lambda x: x.Variable() == Variable.rain, hourly_variables)


hourly_data  = pl.DataFrame({
							'time': pl.datetime_range(
								start=datetime.fromtimestamp(hourly.Time()),
								end=datetime.fromtimestamp(hourly.TimeEnd()),
								time_unit='ms',
								interval='1h',
								closed='left',
								eager=True),
							
						})

for variable in hourly_precipitation:
    member = variable.EnsembleMember()
    hourly_data = hourly_data.with_columns(
        pl.Series(f"precipitation_member{member}", variable.ValuesAsNumpy())
    )

for variable in hourly_rain:
    member = variable.EnsembleMember()
    hourly_data = hourly_data.with_columns(
        pl.Series(f"rain_member{member}", variable.ValuesAsNumpy())
    )


Coordinates 55.125°N 14.875°E
Elevation 20.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


AttributeError: module 'datetime' has no attribute 'fromtimestamp'

In [2]:
import openmeteo_requests
import polars as pl
import datetime
from openmeteo_sdk.Variable import Variable
from openmeteo_sdk.Aggregation import Aggregation
import requests_cache
from retry_requests import retry

class WeatherDataFetcher:
    def __init__(self, latitude, longitude, cache_path='.cache'):
        """
        Initialize the WeatherDataFetcher with coordinates and optional cache path.
        
        Args:
            latitude (float): Latitude of the location
            longitude (float): Longitude of the location
            cache_path (str): Path for the cache file
        """
        self.latitude = latitude
        self.longitude = longitude
        self.url = "https://ensemble-api.open-meteo.com/v1/ensemble"
        
        # Setup cache and client
        cache_session = requests_cache.CachedSession(cache_path, expire_after=3600)
        retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
        self.client = openmeteo_requests.Client(session=retry_session)
        
    def fetch_weather_data(self, start_date, end_date, models="icon_seamless"):
        """
        Fetch weather data for the specified date range.
        
        Args:
            start_date (str): Start date in YYYY-MM-DD format
            end_date (str): End date in YYYY-MM-DD format
            models (str): Weather model to use
            
        Returns:
            tuple: (metadata_dict, polars.DataFrame)
        """
        params = {
            "latitude": self.latitude,
            "longitude": self.longitude,
            "hourly": ["precipitation", "rain"],
            "start_date": start_date,
            "end_date": end_date,
            "models": models
        }
        
        responses = self.client.weather_api(self.url, params=params)
        response = responses[0]
        
        # Extract metadata
        metadata = {
            "coordinates": f"{response.Latitude()}°N {response.Longitude()}°E",
            "elevation": f"{response.Elevation()} m asl",
            "timezone": f"{response.Timezone()} {response.TimezoneAbbreviation()}",
            "utc_offset": f"{response.UtcOffsetSeconds()} s"
        }
        
        # Process hourly data
        hourly = response.Hourly()
        hourly_variables = [hourly.Variables(i) for i in range(hourly.VariablesLength())]
        
        hourly_precipitation = filter(lambda x: x.Variable() == Variable.precipitation, hourly_variables)
        hourly_rain = filter(lambda x: x.Variable() == Variable.rain, hourly_variables)
        
        # Create base DataFrame with time
        df = pl.DataFrame({
            'time': pl.datetime_range(
                start=datetime.datetime.fromtimestamp(hourly.Time()),
                end=datetime.datetime.fromtimestamp(hourly.TimeEnd()),
                time_unit='ms',
                interval='1h',
                closed='left',
                eager=True
            )
        })
        
        # Add precipitation data
        for variable in hourly_precipitation:
            member = variable.EnsembleMember()
            df = df.with_columns(
                pl.Series(f"precipitation_member{member}", variable.ValuesAsNumpy())
            )
        
        # Add rain data
        for variable in hourly_rain:
            member = variable.EnsembleMember()
            df = df.with_columns(
                pl.Series(f"rain_member{member}", variable.ValuesAsNumpy())
            )
            
        return metadata, df
    

In [10]:
# Example usage
fetcher = WeatherDataFetcher(latitude=55.10091, longitude=14.70664)

# Fetch weather data
metadata, df = fetcher.fetch_weather_data(
    start_date="2025-02-12",
    end_date="2025-02-13"
)

In [2]:
from datetime import datetime, timedelta


In [4]:
start_date

'2025-02-14'

In [5]:
end_date

'2025-02-15'